# Clustering & Summarization

### 필요 라이브러리 임포트

In [74]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from collections import Counter
from transformers import *
from summarizer import Summarizer
from sklearn.cluster import KMeans

### 데이터 불러오기

In [244]:
# 최종 기사 선별 후, Call 하기 위해 두가지 버전 의 dataframe 생성

#------ Original------
df = pd.read_csv ('./data/final_df4_rhino.csv')
dforigin = df

#------ High_ Simcount------
dffine = df[df['simcount'] > 4] # 유사한 기사의 수가 4개이상 있는 기사들만 출력 
dffine

,media,date,title,article_original,url,tokenized_sents,sim,simcount
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,트와이스 도쿄돔 공연 블랙핑크도 새앨범 예정경자년 새해에도 K팝 스타들의 세계 무...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['트와이스', '도쿄돔', '공연', '블랙핑크', '앨범', '예정경자', '새...","8, 26, 5, 25, 6, 13, 7",7
14,조선일보,2020-01-15,현대미술과 손 잡은 BTS,미술관서 커넥트 BTS 첫선앤터니 곰리 등 세계적 작가 참여전세계 5개 도시 동시...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['미술관', '첫선앤터니', '세계적', '작가', '참여', '세계', '동시다...","38, 36, 37, 23, 16",5
17,조선일보,2020-01-16,BTS 신보 맵오브더솔7 선주문량 342만장 신기록,방탄소년단BTS이 새 앨범 맵 오브 더 솔7로 다시 한 번 신기록행진을 이어가고 있...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['방탄', '앨범', '오브', '신기록행진', '이번', '신보', '선주문량'...","18, 7, 6, 5, 12",5
19,조선일보,2020-01-17,방탄소년단 새앨범 선공개곡 블랙 스완 깊은 내면의 자전적 가사,그룹 방탄소년단BTS이 다음 달 새 앨범에 발매에 앞서 수록곡 블랙 스완Black ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['그룹', '방탄', '다음', '앨범', '발매', '수록곡', '블랙', '방...","41, 42, 21, 35, 18, 43",6
21,한겨레,2020-01-20,방탄소년단 블랙스완 93개국 아이튠즈 1위K팝 최다,방탄소년단의 정규 4집 선 공개곡 블랙스완이 93개국 아이튠즈 차트 정상을 석권했다...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['방탄소년단', '정규', '공개곡', '블랙스완', '아이', '차트', '정상...","19, 43, 18, 42, 41, 40",6
...,...,...,...,...,...,...,...,...
726,경향신문,2020-12-11,그들은 세상에서 가장 위대한 밴드가 되었다BTS 타임 올해의 연예인으로 선정,경향신문 그룹 방탄소년단BTS이 미국 시사잡지 타임의 2020 올해의 연예인으로 선...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['경향신문', '그룹', '방탄', '미국', '시사잡지', '타임', '올해',...","724, 728, 725, 730, 732, 722, 721",7
730,경향신문,2020-12-11,2020년 타임지가 선정한 올해의 엔터테이너 방탄소년단,경향신문그룹 방탄소년단이 미국 시사주간지 타임의 올해의 엔터테이너로 선정됐다. 타임...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['경향신문그룹', '방탄소년단', '미국', '시사주간지', '타임', '올해',...","732, 728, 724, 726, 725",5
732,조선일보,2020-12-12,타임이 뽑은 올해의 엔터테이너 방탄소년단,K팝 아닌 세계서 가장 큰 밴드 방탄소년단이 미 시사주간지 타임이 선정하는 올해의 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['세계', '밴드', '방탄소년단', '시사주간지', '타임', '올해', '타임...","724, 730, 728, 726, 731, 725",6
734,중앙일보,2020-12-17,코로나BTS부동산SKT 누구가 뽑은 2020 키워드,올해의 키워드는 단연 신종 코로나바이러스 감염증코로나19이었다. 손흥민방탄소년단BT...,https://news.naver.com/main/read.nhn?mode=LSD&...,"['올해', '키워드', '신종', '코로나바이러스', '감염증코', '민방', '...","720, 714, 715, 722, 721",5


### K-means Clustering

In [249]:
testdf = filterkmeans(dffine, 10)

In [252]:
testdf['kmeans_label'].unique()

array([4, 1, 9, 2, 8, 5, 0, 6, 7, 3])

In [245]:
# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def filterkmeans (originaldf, n):
    newser = pd.Series(originaldf['sim'])
    # Series를 데이터 프레임으로 바꾸며, 번호를 전부 찢어준다.
    newdf = pd.DataFrame(newser.str.split(',',9).tolist(), \
                                 columns = ['1','2','3','4','5','6','7','8','9'])
    # 모든 value 'int'화
    newdf = newdf.apply(pd.to_numeric) 
    # drop.na 가 아닌 최소값에 맞춰 불필요한 컬럼을 제거한다.
    newdf = newdf.drop(['6','7','8','9'], axis = 1)
    # kmeans 구간을 n개로 지정
    kmeans = KMeans(int(n))
    kmeans_label = kmeans.fit_predict(newdf)
    newdf['kmeans_label'] = kmeans_label
    
    return newdf

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def getnumlist(originaldf, n):
### filterkmeans 호출해 keep
    newdf = filterkmeans(originaldf, n)
### 다중 리스트 생성 (n개)
### 후에 numlist [index 0~7] 로 접근 가능
    numlist = []
    for i in range(int(n)):
        numlist.append([])
# Kmeans label 번호 정렬
    label_order = []
    for i in newdf['kmeans_label'].unique():
        label_order.append (i)


# label 순으로 1,2,3,4 랭크에 해당되는 숫자들만 list에 append 
    for label_number,sort_number in zip(label_order,range(int(n))):
        certaindf = newdf[newdf['kmeans_label'] == label_number]
        for a in certaindf['1']:
            numlist[sort_number].append(a)
        for b in certaindf['2']:
            numlist[sort_number].append(b)
        for c in certaindf['3']:
            numlist[sort_number].append(c)
#         for d in certaindf['4']:
#             numlist[sort_number].append(d)
#         for e in certaindf['5']:
#             numlist[sort_number].append(e)
            
    return numlist

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------
def listcount (originaldf, n):
    
    # 최종 컬럼들
    medialist = []
    datelist = []
    headlinelist = []
    articlelist = []
    urllist = []
    
    for i in range(int(n)):
        vocab = Counter(getnumlist(originaldf, n)[i])
        samplelist = list(vocab.most_common(1)[0]) # list와 [0]은 tuple 진입을 위해서
        
        media =  dforigin['media'][int(samplelist[0])]
        medialist.append(media)
        
        date = dforigin['date'][int(samplelist[0])]
        datelist.append(date)
        
        headline = dforigin['title'][int(samplelist[0])]
        headlinelist.append(headline)
        
        article_original = dforigin['article_original'][int(samplelist[0])]
        articlelist.append(article_original)
        
        url = dforigin['url'][int(samplelist[0])]
        urllist.append(url)
    
    # 컬럼으로 쓸 data from lists
    data = list(zip(medialist, datelist, headlinelist, articlelist, urllist))
    
    # to Dataframe Columns
    finaldf = pd.DataFrame (data, columns=['media','date','title', 'article_original', 'url'])
    
    
    return finaldf



In [246]:
simple_df = listcount (dffine, 10)
simple_df

,media,date,title,article_original,url
0,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,한겨레,2020-06-04,BTS 인종차별과 폭력에 반대우리 모두 존중받을 권리 있어,그룹 방탄소년단BTS이 인종차별과 폭력에 반대한다는 뜻을 밝혔다. 미국에서 백인 경...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-09-01,방탄소년단 한국 가수 최초 빌보드 싱글 1위K팝 새역사,2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-09-03,빅히트 공모주 청약 앞두고 BTS에 주식 증여상장 시 1인당 최대 200억원대,경향신문방탄소년단BTS이 소속사인 빅히트엔터테인먼트로부터 1인당 7만주 가까운 보통...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인은 아...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,중앙일보,2020-10-14,14억 시장에 휘둘린 BTS로이터 중국은 정치적 지뢰,중국 관영 매체 환구시보가 방탄소년단BTS의 수상 소감이 중국 네티즌의 분노를 일으...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,중앙일보,2020-10-20,택배사 또 BTS제품 거부우리가 했겠나 이유 알지 않나,중국 내 5위 규모의 물류업체 윈다가 방탄소년단BTS 관련 제품 배송을 중단하겠다고...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [239]:
simple_df['article_original'][2]

'그룹 방탄소년단BTS이 인종차별과 폭력에 반대한다는 뜻을 밝혔다. 미국에서 백인 경찰의 무릎에 눌려 숨을 쉴 수 없다고 절규하다 숨진 흑인 조지 플로이드의 죽음에 분노한 인종차별 반대 목소리가 전 세계적으로 번지는 가운데 비티에스도 연대의 목소리를 낸 것이다. 비티에스는 4일 공식 트위터를 통해 우리는 인종차별에 반대합니다. 우리는 폭력에 반대합니다. 나 당신 우리 모두는 존중받을 권리가 있습니다. 함께 하겠습니다라는 글을 한국어와 영어로 올렸다. 이들은 이 글에 BlackLivesMatter흑인의 생명도 중요하다라는 문구를 해시태그로 달기도 했다. 더불어 방탄소년단은 인종차별 반대와 관련한 단체에 기부도 할 예정이다. 소속사 빅히트엔터테인먼트는 기부 단체와 금액은 밝히기 어렵다며 이미 논의를 끝냈고 송금 절차를 진행 중이라고 밝혔다. 지난달 25일 미국 미네소타주 미니애폴리스 파우더호른에서 발생한 조지 플로이드 사건이 알려지면서 미국 전역에서는 이에 항의하는 시위가 번지고 있다. 에스엔에스SNS 등에서도 BlackLivesMatter라는 해시태그를 단 게시물이 잇따르며 인종 차별에 반대하는 목소리가 높아지는 상황이다. 레이디 가가 아리아나 그란데 비욘세 등 팝스타들도 자신의 에스엔에스에 플로이드를 추모하며 인종차별을 비판하는 게시물을 올렸으며 비욘세는 인종차별 반대 청원을 독려하기도 했다. 미국 음악산업계 역시 인종차별 반대의 뜻으로 지난 2일을 블랙아웃 화요일Blackout Tuesday로 정하고 업무를 중단한 바 있다.'

In [247]:
simple_df.to_csv('./data/Willver2_rhino10.csv', index=False)

### Summarization

In [226]:

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('monologg/kobert')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
custom_model = AutoModel.from_pretrained('monologg/kobert', config=custom_config)

summarize_model = Summarizer(custom_model=custom_model, custom_tokenizer = custom_tokenizer)

In [227]:
def summarycolumn (simple_df):
    summarized_list = []
    for i in range(len(simple_df['article_original'])):
        summarized_list.append(summarize_model(simple_df['article_original'][i], ratio= 0.2))
   
    simple_df['summary'] = summarized_list
    
    return simple_df

In [228]:
perfect_df = summarycolumn (simple_df)
perfect_df

,media,date,title,article_original,url,summary
0,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...
1,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...
2,한겨레,2020-06-04,BTS 인종차별과 폭력에 반대우리 모두 존중받을 권리 있어,그룹 방탄소년단BTS이 인종차별과 폭력에 반대한다는 뜻을 밝혔다. 미국에서 백인 경...,https://news.naver.com/main/read.nhn?mode=LSD&...,미국에서 백인 경찰의 무릎에 눌려 숨을 쉴 수 없다고 절규하다 숨진 흑인 조지 플로...
3,조선일보,2020-09-01,방탄소년단 한국 가수 최초 빌보드 싱글 1위K팝 새역사,2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로...,https://news.naver.com/main/read.nhn?mode=LSD&...,2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로...
4,조선일보,2020-09-01,방탄소년단 세계에서 가장 영향력 큰 소년들 됐다 조나스 브라더스 웰컴 투 클럽,한국 가수 최초로 빌보드 차트 싱글 1위 그룹 방탄소년단BTS이 한국 가요 역사를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,한국 가수 최초로 빌보드 차트 싱글 1위 그룹 방탄소년단BTS이 한국 가요 역사를 ...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인은 아...,https://news.naver.com/main/read.nhn?mode=LSD&...,순수예술인은 특례대상인데 대중예술인은 아닌 것에 대한 형평성 문제 사례로 제기한 것...
7,중앙일보,2020-10-13,중국엔 정치적 지뢰 있다BTS 지우기에 깜짝 놀란 외신,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들도...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들도...
8,중앙일보,2020-10-20,택배사 또 BTS제품 거부우리가 했겠나 이유 알지 않나,중국 내 5위 규모의 물류업체 윈다가 방탄소년단BTS 관련 제품 배송을 중단하겠다고...,https://news.naver.com/main/read.nhn?mode=LSD&...,중국 내 5위 규모의 물류업체 윈다가 방탄소년단BTS 관련 제품 배송을 중단하겠다고...
9,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...


In [229]:
for i in perfect_df['summary']:
    print (i, '\n\n')

커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트28일 서울 개막 강이연 등 참여 인종세대 다른 아미에게 영감곰리가 참여한 뉴욕전 내달 개최 일곱 명의 그림자가 장막 뒤에 있다. 한국예술종합학교 신창호 교수가 짠 안무를 1학년 학생들에게 맡긴 뒤 촬영했다. 여러 나라와 기관과 작가가 BTS란 축으로 연결되는 새 플랫폼이 창출됐다. 개별 작품도 좋지만 다섯 도시의 점이 이어져 어떤 그림이 될지 살핀다면 더 흥미로울 것이다. 


그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영국 오피셜 차트에 이어 빌보드까지 양대 팝 차트 1위에 올랐다. 빌보드가 인용한 닐슨뮤직 데이터의 자료를 보면 지난달 21일 발매된 이 앨범은 27일까지 총 42만2000장의 판매고를 올렸다. 이외에 스트리밍 횟수를 앨범 판매량으로 환산한 수치가 4만8000장 디지털 음원 다운로드 횟수를 앨범 판매량으로 환산한 수치는 2만6000장으로 집계됐다. 이후 2018년 9월 러브 유어셀프 결 앤서 지난해 4월 맵 오브 더 솔 페르소나에 이어 이번 앨범까지 1년9개월 만에 4회 연속 빌보드 200 1위를 차지했다. 


미국에서 백인 경찰의 무릎에 눌려 숨을 쉴 수 없다고 절규하다 숨진 흑인 조지 플로이드의 죽음에 분노한 인종차별 반대 목소리가 전 세계적으로 번지는 가운데 비티에스도 연대의 목소리를 낸 것이다. 


2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로 빌보드 메인 싱글 차트인 핫 100 정상에 오르며 K팝의 새 역사를 썼다. 지난 2012년 세계적으로 히트한 싸이의 강남스타일이 7주 연속 2위를 기록한 적이 있지만 1위에는 오르지 못했다. 다이너마이트는 앞서 2주 연속으로 핫 100 1위를 차지했던 여성래퍼 카디 비와 메건 더 스탤리언의 WAP를 제쳤다. 특히 디지털 음원 다운로드가 26만5000여건으로 지난 2017년 9월 테일러 스위프트의 룩 왓 유 메이드 미 두 이래 약 3년 만에